In [2]:
from mlflow import MlflowClient

mlflow_client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [3]:
run_178 = "d5fc2d434f3c4c58aff9b2a5f02a9b0a"
run_179 = "9540f6dc8c5743a88514533a2c354240"
trial_num = 179
run_info = mlflow_client.get_run(run_179)

In [4]:
from drecg.models.feat_extraction import DiffFeatureDetectorParam
params = run_info.data.params
model = DiffFeatureDetectorParam(num_hidden=int(params['num_hidden']),
                   hidden_units=int(params['hidden_units']),
                   features_dropout=float(params['features_dropout']),
                   hidden_dropout=float(params['hidden_dropout']),
                   hidden_act=params['hidden_act'],
                   features_dim=int(params['features_size']))

In [20]:
import torch
from drecg.models.feat_extraction import DiffFeatureDetectorParamBiDirectional
params = run_info.data.params
trial_num = 62
run_62 = "720c3339f754418683b69e5c521f3656"
run_info = mlflow_client.get_run(run_62)
params = run_info.data.params
model_62 = DiffFeatureDetectorParamBiDirectional(num_hidden=int(params['num_hidden']),
                   hidden_units=int(params['hidden_units']),
                   features_dropout=float(params['features_dropout']),
                   hidden_dropout=float(params['hidden_dropout']),
                   hidden_act=params['hidden_act'],
                   features_dim=int(params['features_size']))

model_62.load_state_dict(torch.load(f"saved_models/optim_10_feb_2023/{trial_num}.ckpt"))

<All keys matched successfully>

In [5]:
import torch
model.load_state_dict(torch.load(f"mlflow_models/{trial_num}.ckpt"))

<All keys matched successfully>

In [6]:

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted_first_deploy.pt')

In [47]:
model_scripted = torch.jit.script(model_62) # Export to TorchScript
model_scripted.save('model_scripted_62.pt') # Save
model_scripted

RecursiveScriptModule(
  original_name=DiffFeatureDetectorParamBiDirectional
  (cls_layer): RecursiveScriptModule(
    original_name=Sequential
    (features_dropout): RecursiveScriptModule(original_name=Dropout)
    (linear_0): RecursiveScriptModule(original_name=Linear)
    (relu_0): RecursiveScriptModule(original_name=ReLU)
    (hidden_dropout_0): RecursiveScriptModule(original_name=Dropout)
    (linear_1): RecursiveScriptModule(original_name=Linear)
    (relu_1): RecursiveScriptModule(original_name=ReLU)
    (hidden_dropout_1): RecursiveScriptModule(original_name=Dropout)
    (linear_out): RecursiveScriptModule(original_name=Linear)
  )
)

In [7]:
model2 = torch.jit.load('model_scripted.pt')
model2.eval()

RecursiveScriptModule(
  original_name=DiffFeatureDetectorParam
  (cls_layer): RecursiveScriptModule(
    original_name=Sequential
    (features_dropout): RecursiveScriptModule(original_name=Dropout)
    (linear_0): RecursiveScriptModule(original_name=Linear)
    (relu_0): RecursiveScriptModule(original_name=ReLU)
    (hidden_dropout_0): RecursiveScriptModule(original_name=Dropout)
    (linear_out): RecursiveScriptModule(original_name=Linear)
  )
)

In [27]:
import PIL.Image as Image
def read_sample_imgs(num, path='./samples'):
    img_a = Image.open(f"{path}/{num}a.png")
    img_b = Image.open(f"{path}/{num}b.png")
    img_a, img_b = img_a.convert('RGB'), img_b.convert('RGB')
    return img_a, img_b


In [28]:
from drecg.models.feat_extraction import VitLaionFeatureExtractor
feat_extractor = VitLaionFeatureExtractor()
processor = feat_extractor.transforms

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
import torch
model = model_scripted
model.eval()
feat_extractor.to(device)
model.to(device)
def predict_diff(sample_num):
    a, b = read_sample_imgs(sample_num)
    x = processor(a).unsqueeze(dim=0).to(device), processor(b).unsqueeze(dim=0).to(device)
    feats = feat_extractor(x)
    proba = torch.sigmoid(model((feats[0].to(device), feats[1].to(device)))).item()
    print("%.7f" % proba)

In [49]:
for i in [1,3,4,5,6,8,28, 10, 11, 12 ,13, 20, 21,22,23,24,25,26,27,28]:
    print(i)
    predict_diff(i)

1
0.0000023
3
0.9970760
4


/home/daniel/mambaforge/envs/dogrec/lib/python3.10/site-packages/torch/nn/modules/module.py:1194: UserWarning: FALLBACK path has been taken inside: runCudaFusionGroup. This is an indication that codegen Failed for some reason.
To debug try disable codegen fallback path via setting the env variable `export PYTORCH_NVFUSER_DISABLE=fallback`
 (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541990/work/torch/csrc/jit/codegen/cuda/manager.cpp:331.)
  return forward_call(*input, **kwargs)


0.2090216
5
0.1683520
6
0.9458011
8
0.9732897
28
0.0026034
10
0.0000000
11
0.0000033
12
0.0080022
13
0.0735051
20
0.0053240
21
0.0000000
22
0.7028679
23
0.8426622
24
0.0032309
25
0.9177830
26
0.0544644
27
0.7434675
28
0.0026034


In [46]:
for i in [1,3,4,5,6,8,28, 10, 11, 12 ,13, 20, 21,22,23,24,25,26,27,28]:
    print(i)
    predict_diff(i)

1
0.0000008
3
0.9989937
4
0.7613308
5
0.1346551
6
0.9616435
8
0.9964180
28
0.1397899
10
0.0000001
11
0.0000000
12
0.0009009
13
0.0031633
20
0.0040430
21
0.0000000
22
0.8266684
23
0.9964795
24
0.0002980
25
0.9864895
26
0.0014194
27
0.1015408
28
0.1397899


In [45]:
model

DiffFeatureDetectorParam(
  (cls_layer): Sequential(
    (features_dropout): Dropout(p=0.43686443329650726, inplace=False)
    (linear_0): Linear(in_features=1280, out_features=110, bias=True)
    (relu_0): ReLU()
    (hidden_dropout_0): Dropout(p=0.46546498086133914, inplace=False)
    (linear_out): Linear(in_features=110, out_features=1, bias=True)
  )
)

In [24]:
from torch.utils.data import DataLoader
from drecg.data.utils import FeaturesDataset

test_dataset_bi = FeaturesDataset('features_ext_vit_laion/validation_features.pt', bidirectional=True)
test_dataset = FeaturesDataset('features_ext_vit_laion/validation_features.pt', bidirectional=False)
test_dataloader_bi = DataLoader(test_dataset_bi, batch_size=32, shuffle=False, num_workers=12)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=12)

In [6]:
from ignite.engine import create_supervised_evaluator
from drecg.training.ignite import acc_transform_fn, output_transform_fn, prepare_batch_fn
from ignite.metrics import Accuracy, Loss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = torch.nn.BCEWithLogitsLoss()
model.to(device)

val_metrics = {
    "accuracy": Accuracy(output_transform=acc_transform_fn),
    "loss": Loss(loss_fn, output_transform=lambda out: (out[1], out[0]))}
evaluator = create_supervised_evaluator(
        model,
        device=device,
        output_transform=output_transform_fn,
        prepare_batch=prepare_batch_fn,
        non_blocking=True,
        metrics=val_metrics)

In [7]:
evaluator.run(test_dataloader)
evaluator.state.metrics

{'accuracy': 0.8872832369942196, 'loss': 0.3138719679992323}

In [8]:
evaluator.run(test_dataloader_bi)
evaluator.state.metrics

{'accuracy': 0.880057803468208, 'loss': 0.30866464162837565}

In [21]:
model_62.to(device)
val_metrics = {
    "accuracy": Accuracy(output_transform=acc_transform_fn),
    "loss": Loss(loss_fn, output_transform=lambda out: (out[1], out[0]))}
evaluator = create_supervised_evaluator(
        model_62,
        device=device,
        output_transform=output_transform_fn,
        prepare_batch=prepare_batch_fn,
        non_blocking=True,
        metrics=val_metrics)

In [17]:
evaluator.run(test_dataloader)
evaluator.state.metrics

{'accuracy': 0.9132947976878613, 'loss': 0.27019147928050485}

In [18]:
evaluator.run(test_dataloader_bi)
evaluator.state.metrics

{'accuracy': 0.9132947976878613, 'loss': 0.27019147928050485}

In [22]:
evaluator.run(test_dataloader)
evaluator.state.metrics

{'accuracy': 0.9219653179190751, 'loss': 0.23985857219365292}

In [23]:
evaluator.run(test_dataloader_bi)
evaluator.state.metrics

{'accuracy': 0.9219653179190751, 'loss': 0.2398585942439261}

In [26]:
evaluator.run(test_dataloader_bi)
evaluator.state.metrics

{'accuracy': 0.9512195121951219, 'loss': 0.12108118941144246}